In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.stattools import adfuller
import pickle



In [ ]:
df = pd.read_parquet("../data/processed/ENEFIT_dataset.parquet")
df['hour'] = df.index.hour
df['month'] = df.index.month
df['day'] = df.index.weekday + 1
df['target_production'] = df['installed_capacity'] * df['direct_solar_radiation'] / (df['temperature'] + 273.15)
df["capacity_per_eic"] = np.round(df["installed_capacity"] / df["eic_count"], 2)
df['Weekday'] = [0 if day > 4 else 1 for day in df.index.dayofweek]
df['Weekday'] = df['Weekday'].astype('int64')  # Explicitly cast to int64


In [ ]:
def split_series(df, n_past, n_future, target_column_name, feature_column_names):
    """
    Split a DataFrame into past features and future target arrays.

    Parameters:
    - df: DataFrame containing the time series data.
    - n_past: Number of past observations to use for predicting the future.
    - n_future: Number of future observations to predict.
    - target_column_name: Name of the target column.
    - feature_column_names: List of column names to be used as features.
    - scaling: if dataset is scaled REMOVED

    Returns:
    - X: Array of past observations' features.
    - y: Array of future observations' target values.
    Only if scaling is true:
    - feature_scaler: scaler for X REMOVED
    - target_scaler: scaler for y REMOVED
    """

    # if scaling == 1:
    #     feature_scaler = MinMaxScaler()
    #     target_scaler = MinMaxScaler()
        
    #     # Fit the scalers
    #     feature_scaler.fit(df[feature_column_names])
    #     target_scaler.fit(df[[target_column_name]])
        
    #     # Apply the transformations
    #     scaled_features = feature_scaler.transform(df[feature_column_names])
    #     scaled_target = target_scaler.transform(df[[target_column_name]])

    X, y = list(), list()
    for window_start in range(len(df)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(df):
            break
        # Select the columns by name for the past and future segments
        # if scaling == 1:
        #     past = scaled_features[window_start:past_end]
        #     future = scaled_target[past_end:future_end]
        past = df.iloc[window_start:past_end][feature_column_names].values
        future = df.iloc[past_end:future_end][target_column_name].values
        X.append(past)
        y.append(future)
    X, y = np.array(X), np.array(y)
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    # if scaling == 1:
    #     return X, y, feature_scaler, target_scaler
    return X, y

def reformat_predictions_actual(pred, org_X_train):
       '''
       Converts the diff predictions into the original values, returns actual predictions and original values
       y_train[1:,:] + org_y_train[0:-1,:] == org_y_train[1]
       '''
       final_predictions = np.zeros_like(pred)

       for i in range(pred.shape[0]):
              final_predictions[i, 0] = org_X_train[i,-1] + pred[i,0]
              for j in range(1, pred.shape[1]):
                     final_predictions[i, j] = final_predictions[i, j-1] + pred[i, j]
       return final_predictions

In [ ]:

n_past = 24
n_future = 24
target_column_name = 'diff_demand'
train_df, val_df, test_df = df[1:int(len(df)*0.7)], df[int(len(df)*0.7):int(len(df)*0.7)+int(len(df)*0.15)], df[int(len(df)*0.7)+int(len(df)*0.15):] 

ARIMA_X_train, ARIMA_y_train = split_series(train_df,n_past, n_future, target_column_name, target_column_name)
ARIMA_X_val, ARIMA_y_val = split_series(val_df,n_past, n_future, target_column_name, target_column_name)
ARIMA_X_test, ARIMA_y_test = split_series(test_df,n_past, n_future, target_column_name, target_column_name)
org_X_train, org_y_train = split_series(train_df,n_past, n_future, 'demand', 'demand')
org_X_val, org_y_val = split_series(val_df,n_past, n_future, 'demand', 'demand')
org_X_test, org_y_test = split_series(test_df,n_past, n_future, 'demand', 'demand')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# PACF for p
plot_pacf(train_df[target_column_name], lags=30)
plt.title("Partial Autocorrelation for Demand")
plt.show()

# ACF for q
plot_acf(train_df[target_column_name], lags=30)
plt.title("Autocorrelation for Demand")
plt.show()


In [ ]:
model = auto_arima(train_df[target_column_name], start_p=0, start_q=0,
                            test='adf',       
                            max_p=30, max_q=30, 
                            m=1,              
                            d=None,           
                            seasonal=False,   
                            start_P=0, 
                            D=0, 
                            trace=False,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

print(model.summary())

#Note: auto_arima uses a SARIMAX, but because I don't add any exogenous variables and seasonality is false, it is still an ARIMA model

#6,0,5

In [ ]:
from tqdm import tqdm

def train_predict_ARIMA(train, steps, model):
    forecast = []
    forecast.append(model.predict(n_periods=steps))
    for i in tqdm(range(1, train.shape[0])):
        model.update(train[i-1,-1])
        forecast.append(model.predict(n_periods=steps))
    return np.array(forecast), model

In [ ]:
# train_forecast, model = train_predict_ARIMA(ARIMA_X_train, 24, None)
val_pred, model = train_predict_ARIMA(ARIMA_X_val, 24, model)
test_pred, model = train_predict_ARIMA(ARIMA_X_test, 24, model)

In [ ]:
arima_y_val_pred = reformat_predictions_actual(val_pred, org_X_val)
arima_y_test_pred = reformat_predictions_actual(test_pred, org_X_test)

In [ ]:
arima_results = {"train": None, "val": val_pred, "test": test_pred, "formatted_train": None, "formatted_val":arima_y_val_pred, "formatted_test": arima_y_test_pred}

In [ ]:
with open("../results/demand_arima_results.pkl", "wb") as outfile: 
    pickle.dump(arima_results, outfile)

In [ ]:

n_past = 24
n_future = 24
target_column_name = 'diff_supply'
train_df, val_df, test_df = df[1:int(len(df)*0.7)], df[int(len(df)*0.7):int(len(df)*0.7)+int(len(df)*0.15)], df[int(len(df)*0.7)+int(len(df)*0.15):] 

ARIMA_X_train, ARIMA_y_train = split_series(train_df,n_past, n_future, target_column_name, target_column_name)
ARIMA_X_val, ARIMA_y_val = split_series(val_df,n_past, n_future, target_column_name, target_column_name)
ARIMA_X_test, ARIMA_y_test = split_series(test_df,n_past, n_future, target_column_name, target_column_name)
org_X_train, org_y_train = split_series(train_df,n_past, n_future, 'supply', 'supply')
org_X_val, org_y_val = split_series(val_df,n_past, n_future, 'supply', 'supply')
org_X_test, org_y_test = split_series(test_df,n_past, n_future, 'supply', 'supply')

In [ ]:
# PACF for p
plot_pacf(train_df[target_column_name], lags=30)
plt.title("Partial Autocorrelation for Demand")
plt.show()

# ACF for q
plot_acf(train_df[target_column_name], lags=30)
plt.title("Autocorrelation for Demand")
plt.show()


In [ ]:
model = auto_arima(train_df[target_column_name], start_p=0, start_q=0,
                            test='adf',       
                            max_p=30, max_q=30, 
                            m=1,              
                            d=None,           
                            seasonal=False,   
                            start_P=0, 
                            D=0, 
                            trace=False,
                            error_action='ignore',  
                            suppress_warnings=True, 
                            stepwise=True)

print(model.summary())

#Note: auto_arima uses a SARIMAX, but because I don't add any exogenous variables and seasonality is false, it is still an ARIMA model

#6,0,5

In [ ]:
# train_forecast, model = train_predict_ARIMA(ARIMA_X_train, 24, None)
val_pred, model = train_predict_ARIMA(ARIMA_X_val, 24, model)
test_pred, model = train_predict_ARIMA(ARIMA_X_test, 24, model)

In [ ]:
arima_y_val_pred = reformat_predictions_actual(val_pred, org_X_val)
arima_y_test_pred = reformat_predictions_actual(test_pred, org_X_test)

In [ ]:
arima_results = {"train": None, "val": val_pred, "test": test_pred, "formatted_train": None, "formatted_val":arima_y_val_pred, "formatted_test": arima_y_test_pred}

In [ ]:
with open("../results/supply_arima_results.pkl", "wb") as outfile: 
    pickle.dump(arima_results, outfile)